In [1]:
import os

# # import matplotlib
# matplotlib.use('agg')
# # import matplotlib
# matplotlib.use('agg')

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from os.path import dirname, exists, join as pjoin
from os import makedirs, listdir
import pandas
import seaborn as sns
import pandas as pd
import matplotlib.patches as patches
import mne
from tqdm.auto import tqdm
from TorchSpectra import compute_spectrogram
from matplotlib.ticker import FormatStrFormatter

# use pd.set_options() to reduce the size of a pandas dataframe
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.precision', 3)

# Patient by patient (folder by folder manually)

In [238]:
patient = 'jh105' # Specify this every time
file = 'sub-jh105_ses-presurgery_task-ictal_acq-ecog_run-05' # Specify this every time

# Read VHDR EEG file
folder_path = f'E:\\{patient}\\3_segmented'
eeg_file_name =  f'{file}_ieeg.vhdr'
file_path = folder_path + '\\' + eeg_file_name
MNE_data = mne.io.read_raw_brainvision(file_path)
channels_MNE = MNE_data.ch_names

# Read TSV file for channels information
tsv_file_name =  f'{file}_channels.tsv'
file_path = folder_path + '\\' + tsv_file_name
tsv = pandas.read_csv(file_path,sep='\t')
fs = tsv["sampling_frequency"][0]
status = tsv["status"]
bad_channels_idxs = np.where(status != 'good')[0]

# Add channels that start with $ as bad channels too
for i in range(len(channels_MNE)):
    if channels_MNE[i][0] == '$':
        if i not in bad_channels_idxs.tolist():
            bad_channels_idxs = bad_channels_idxs.tolist()
            bad_channels_idxs.append(i)
            bad_channels_idxs = np.array(bad_channels_idxs)

# Drop and delete bad channels from EEG MNE
bad_channels = [channels_MNE[chi_idx] for chi_idx in bad_channels_idxs]
MNE_data.drop_channels(bad_channels)
channels_MNE = MNE_data.ch_names

Extracting parameters from E:\jh105\3_segmented\sub-jh105_ses-presurgery_task-ictal_acq-ecog_run-05_ieeg.vhdr...
Setting channel info structure...


In [239]:
# Load a sample .MAT file for the reference
tmp_folder_path = 'G:\\ieeg\\cc\\epilepsy_data_processed\\sub-la24\\ieeg\\data\\ses-presurgery' # Don't need to change every time
tmp_eeg_file_name =  'sub-la24_ses-presurgery_task-interictal_acq-seeg_run-01_ieeg.mat' # Don't need to change every time
tmp_file_path = tmp_folder_path + '\\' + tmp_eeg_file_name

eeg = sio.loadmat(tmp_file_path)['pt_data'][0]
eeg['labels'][0] = [chi for chi in channels_MNE] #np.array([[[chi]] for chi in channels_MNE])
eeg['ez_ch'][0] = np.array([])
eeg['resect_ch'][0] = np.array([])
eeg['ablate_ch'][0] = np.array([])
eeg['fs'][0] = np.array([[int(fs)]])
eeg['center'][0] = np.array([])
eeg['engel_score'][0] = np.array([])
eeg['ilae_score'][0] = np.array([])
eeg['outcome'][0] = np.array([])
eeg["data_filt"][0] = MNE_data.get_data()
eeg["data_filt_car"][0] = MNE_data.get_data()

# Save the mne MAT file
def structured_to_dict(arr: np.ndarray):
    return {varname: arr[varname] for varname in eeg.dtype.names}
eeg_file_name =  f'{file}_ieeg.mat'
file_path = folder_path + '\\' + eeg_file_name
sio.savemat(file_path, structured_to_dict(eeg))

# All patients from Open Neuro together

In [8]:
# Specify the path to your Excel file
excel_file_path = 'results_db.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

# Filter rows where the 'Storage' column is 'Open Neuro'
filtered_df = df[df['Storage'] == 'Open Neuro']

# Extract the 'Name' column values from the filtered DataFrame
names_from_open_neuro = filtered_df['Name'].tolist()

# Display the extracted names
print(names_from_open_neuro)

for patienti in names_from_open_neuro:
    folder_path = f'E:\\{patienti}\\3_segmented'

    # Find all the segments available for the patient
    segments = []
    for filei in listdir(folder_path):
        seg = filei.split(".")[0]
        seg = "_".join(seg.split("_")[:-1])
        if seg not in segments: segments.append(seg)

    # Run the code for each segment, one by one
    for seg in segments:

        # Read VHDR EEG file for EEG data
        eeg_file_name = seg + '_ieeg.vhdr'
        file_path = folder_path + '\\' + eeg_file_name
        print(file_path)
        MNE_data = mne.io.read_raw_brainvision(file_path)
        channels_MNE = MNE_data.ch_names

        # Read TSV file for channels information
        tsv_file_name =  f'{seg}_channels.tsv'
        file_path = folder_path + '\\' + tsv_file_name
        tsv = pandas.read_csv(file_path,sep='\t')
        fs = tsv["sampling_frequency"][0]
        status = tsv["status"]
        bad_channels_idxs = np.where(status != 'good')[0]

        # Add channels that start with $ as bad channels too
        for i in range(len(channels_MNE)):
            if channels_MNE[i][0] == '$':
                if i not in bad_channels_idxs.tolist():
                    bad_channels_idxs = bad_channels_idxs.tolist()
                    bad_channels_idxs.append(i)
                    bad_channels_idxs = np.array(bad_channels_idxs)

        # Drop and delete bad channels from EEG MNE
        bad_channels = [channels_MNE[chi_idx] for chi_idx in bad_channels_idxs]
        MNE_data.drop_channels(bad_channels)
        channels_MNE = MNE_data.ch_names

        # Load a sample .MAT file for the reference, we then put the data into it
        tmp_folder_path = 'G:\\ieeg\\cc\\epilepsy_data_processed\\sub-la24\\ieeg\\data\\ses-presurgery' # Don't need to change every time
        tmp_eeg_file_name =  'sub-la24_ses-presurgery_task-interictal_acq-seeg_run-01_ieeg.mat' # Don't need to change every time
        tmp_file_path = tmp_folder_path + '\\' + tmp_eeg_file_name

        eeg = sio.loadmat(tmp_file_path)['pt_data'][0]
        eeg['labels'][0] = [chi for chi in channels_MNE] #np.array([[[chi]] for chi in channels_MNE])
        eeg['ez_ch'][0] = np.array([])
        eeg['resect_ch'][0] = np.array([])
        eeg['ablate_ch'][0] = np.array([])
        eeg['fs'][0] = np.array([[int(fs)]])
        eeg['center'][0] = np.array([])
        eeg['engel_score'][0] = np.array([])
        eeg['ilae_score'][0] = np.array([])
        eeg['outcome'][0] = np.array([])
        eeg["data_filt"][0] = MNE_data.get_data()
        eeg["data_filt_car"][0] = MNE_data.get_data()

        # Save the mne MAT file
        def structured_to_dict(arr: np.ndarray):
            return {varname: arr[varname] for varname in eeg.dtype.names}
        eeg_file_name =  f'{seg}_ieeg.mat'
        file_path = folder_path + '\\' + eeg_file_name
        sio.savemat(file_path, structured_to_dict(eeg))

    print("=====================================================================")
    print(F"+++++ {patienti:5s} DONE +++++++++++++++++++++++++++++++++++++++++++++++")
    print("=====================================================================")


['jh102', 'jh104', 'jh106', 'ummc001', 'ummc007', 'jh105', 'pt01', 'pt02', 'pt03', 'pt08', 'pt11', 'pt13', 'pt15', 'pt16', 'pt17', 'umf001', 'umf004', 'umf005', 'ummc002', 'ummc003', 'ummc004', 'ummc005', 'ummc006', 'ummc008', 'ummc009', 'pt06', 'pt10', 'pt12', 'umf002', 'pt07', 'umf003', 'jh101', 'jh103', 'jh108', 'pt14']
E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...
E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr
Extracting parameters from E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr...
Setting channel info structure...
E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr
Extracting parameters from E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr...
Setting channel inf

C:\Users\adaraie\AppData\Local\Temp\ipykernel_54616\3659007579.py:33: RuntimeWarning: Omitted 2 annotation(s) that were outside data range.
  MNE_data = mne.io.read_raw_brainvision(file_path)


+++++ umf004 DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\umf005\3_segmented\sub-umf005_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\umf005\3_segmented\sub-umf005_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...


C:\Users\adaraie\AppData\Local\Temp\ipykernel_54616\3659007579.py:33: RuntimeWarning: Omitted 2 annotation(s) that were outside data range.
  MNE_data = mne.io.read_raw_brainvision(file_path)


+++++ umf005 DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...
E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr
Extracting parameters from E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr...
Setting channel info structure...
E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr
Extracting parameters from E:\ummc002\3_segmented\sub-ummc002_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr...
Setting channel info structure...
+++++ ummc002 DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\ummc003\3_segmented\sub-ummc003_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\ummc003\3_segmented\sub-ummc003_ses-

C:\Users\adaraie\AppData\Local\Temp\ipykernel_54616\3659007579.py:33: RuntimeWarning: Omitted 2 annotation(s) that were outside data range.
  MNE_data = mne.io.read_raw_brainvision(file_path)


+++++ umf002 DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...
E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr
Extracting parameters from E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr...
Setting channel info structure...
E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr
Extracting parameters from E:\pt07\3_segmented\sub-pt7_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr...
Setting channel info structure...
+++++ pt07  DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\umf003\3_segmented\sub-umf003_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\umf003\3_segmented\sub-umf003_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr.

C:\Users\adaraie\AppData\Local\Temp\ipykernel_54616\3659007579.py:33: RuntimeWarning: Omitted 2 annotation(s) that were outside data range.
  MNE_data = mne.io.read_raw_brainvision(file_path)


+++++ umf003 DONE +++++++++++++++++++++++++++++++++++++++++++++++
E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr
Extracting parameters from E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...
E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr
Extracting parameters from E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.vhdr...
Setting channel info structure...
E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr
Extracting parameters from E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.vhdr...
Setting channel info structure...
E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-04_ieeg.vhdr
Extracting parameters from E:\jh101\3_segmented\sub-jh101_ses-presurgery_task-ictal_acq-ecog_run-04_ieeg.vhdr...
Setting channel info structure...
+++++ jh101 DO

In [127]:
# # Read the good channels from Adam's files
#

# data_eeg = eeg['data_filt'][0]
# eeg_labels = eeg['labels'][0]
# channels_MAT = []
# for i in range(len(eeg_labels)):
#     channels_MAT.append(eeg_labels[i][0][0])

# channels_EEG_missed = []
# for chi in channels_MNE:
#     if chi not in channels_MAT:
#         channels_EEG_missed.append(chi)
# print(channels_EEG_missed)
#
# # Delete channels from MNE
# MNE_data.drop_channels(channels_EEG_missed)

# Add all the patients to the processing_db.xlsx sheet from Notion

In [57]:
# Useful utility functions
def keep_numeric(string):
    return ''.join(c for c in string if c.isdigit())
def keep_letters(string):
    return ''.join(c for c in string if not c.isdigit())
def channels_str_to_list(cell_str):
    if len(cell_str) == 0:
        return ['']

    contacts = []
    for contacti in cell_str.split(","):
        contacti = contacti.strip()
        if "-" in contacti:
            ch_num_1,ch_num_2 = contacti.split("-")
            ch_reg = keep_letters(ch_num_1)
            ch_num_1, ch_num_2 = keep_numeric(ch_num_1), keep_numeric(ch_num_2)
            for contactj in range(int(ch_num_1), int(ch_num_2)+1):
                contacts.append(f"{ch_reg}{contactj}")
        else:
            ch_num = keep_numeric(contacti)
            ch_reg = keep_letters(contacti)
            contacts.append(f"{ch_reg}{ch_num}")

    return contacts
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

In [122]:
excel_file_path = 'results_db.xlsx' # Specify the path to your Excel file
df = pd.read_excel(excel_file_path) # Read the Excel file into a pandas DataFrame
filtered_df = df[df['Storage'] == 'Open Neuro'] # Filter rows where the 'Storage' column is 'Open Neuro'
names_from_open_neuro = filtered_df['Name'].tolist() # Extract the 'Name' column values from the filtered DataFrame
print(names_from_open_neuro) # Display the extracted names

excel_file_path = 'processing_db.xlsx' # Specify the path to your Excel file
df = pd.read_excel(excel_file_path) # Read the Excel file into a pandas DataFrame
names_from_processing_db = df['patient_no'].tolist() # Extract the 'Name' column values from the filtered DataFrame
print(names_from_processing_db) # Display the extracted names

for namei_from_open_neuro in names_from_open_neuro:
    if namei_from_open_neuro not in names_from_processing_db: # If the name was not in the processing db, then add it
        print("=================================================")
        print(f"++++++++++ Adding {namei_from_open_neuro} ++++++++++")
        print("=================================================")
        reference_row = df.iloc[0].copy() # copy the first row as a reference
        reference_row["patient_no"] = namei_from_open_neuro
        reference_row["ents_std"] = 0.00906874
        reference_row["ents_med"] = 0.896192635
        reference_row["electrode_colors"] = np.nan
        reference_row["coeff"] = 6
        reference_row["th_low"] = 100
        reference_row["th_high"] = np.nan
        reference_row["th_0"] = 2500
        reference_row["n_types to find noise"] = 2
        reference_row["excluded noise cluster"] = 'x'
        reference_row["n_clusters excluding noise"] = 1
        reference_row["soz localization groups"] = "[[1]]"
        reference_row["contacts to eliminate noise cluster"] = np.nan

        # Loading SOZs from the file
        openNeuro_df = pd.read_excel("E:\\OpenNeuro clinical_data_summary.xlsx") # Read the Excel file into a pandas DataFrame
        patient_openNeuro_row = openNeuro_df[openNeuro_df["dataset_id"] == namei_from_open_neuro]

        sozs = patient_openNeuro_row["soz_contacts"].values[0]
        # process sozs
        if type(sozs) != str and np.isnan(sozs):
            continue

        for i, chari in enumerate(sozs):
            if chari in [";", ","]:
                sozs = replace_str_index(text=sozs,index=i,replacement=" ")  # replace ; or , with " "
        sozs = ",".join(sozs.split()) # then join them with comman ,
        sozs = sozs.upper()

        # process bad channels
        bads = patient_openNeuro_row["bad_contacts"].values[0]
        if not (type(bads) != str and np.isnan(bads)):
            for i, chari in enumerate(bads):
                if chari in [";", ","]:
                    bads = replace_str_index(text=bads,index=i,replacement=" ")  # replace ; or , with " "
            bads = ",".join(bads.split()) # then join them with comman ,
            bads = ",".join(channels_str_to_list(bads)) # expland them and then join them as string
            bads = bads.upper()
        else:
            bads = ""

        reference_row["soz_contacts"] = "[[" + sozs + "]]"
        reference_row["spread_contacts"] = np.nan
        reference_row["ablated_contacts"] = np.nan
        reference_row["outliers_labels"] = bads

        # Add the row
        df.loc[len(df)] = reference_row


['jh102', 'jh104', 'jh106', 'ummc001', 'ummc007', 'jh105', 'pt01', 'pt02', 'pt03', 'pt08', 'pt11', 'pt13', 'pt15', 'pt16', 'pt17', 'umf001', 'umf004', 'umf005', 'ummc002', 'ummc003', 'ummc004', 'ummc005', 'ummc006', 'ummc008', 'ummc009', 'pt06', 'pt10', 'pt12', 'umf002', 'pt07', 'umf003', 'jh101', 'jh103', 'jh108', 'pt14']
['PY23N001', 'PY23N009', 'PY23N011', 'PY23N017', 'PY23N018', 'PY23N019', 'PY23N021', 'PY23N024', 'PY23N025', 'PY23N027', 'PY23N030', 'PY23N032', 'PY23N033', 'PY23N034', 'PY23N035', 'ID01', 'ID02', 'ID03', 'ID04', 'ID05', 'ID06', 'ID07', 'ID08', 'ID09', 'ID10', 'ID11', 'ID12', 'ID13', 'ID14', 'ID15', 'ID16', 'ID17', 'ID18', 'jh103', 'la01', 'la02', 'la04', 'la05', 'la06', 'la07', 'la08', 'la09', 'la11', 'la13', 'la15', 'la16', 'la21', 'la22', 'la23', 'nl04', 'nl05', 'nl07', 'nl08', 'nl13', 'nl15', 'nl16', 'nl18', 'la24', 'nl14', 'la03', 'la10', 'jh105']
++++++++++ Adding jh102 ++++++++++
++++++++++ Adding jh104 ++++++++++
++++++++++ Adding jh106 ++++++++++
++++++++++ 

In [123]:
# df.to_excel("processing_db_v2.xlsx")

# Fill the electrode color column in processing_db for each patient using the channels for them

Example: {"RAT":"#Ff0000", "RPP":"#Ff8c00", "LPT":"#5cbb38", "LPP":"#44a3d5", "LSI":"#005aff", "RPT":"#9d00ff", "LST":"#D944e0", "RIP":"#Ff007e", "RSI":"#30812b", "LIP":"#6f812b", "LAT":"#643fa6", "RIF":"#A63f7b", "LDI":"#4f527d", "LBT":"#0583F2", "LIF":"#F24405", "RAF":"#BF5454", "RBT":"#03588C", "RST":"#025940", "LAF":"#026873", "RDI":"#023873"}


In [157]:
def split_label(s):
    # Put the eeg data into bipolar format
    region = s.rstrip('0123456789')
    number = s[len(region):]
    return region, int(number)


In [156]:
lists_patients = ["jh102", "jh104", "ummc001", "ummc007", "pt01", "pt02", "pt03", "pt08", "pt11", "pt13", "pt15", "pt16", "pt17", "umf001", "umf004", "umf005", "ummc002", "ummc003", "ummc004", "ummc005", "ummc006", "ummc008", "ummc009", "pt06", "pt10", "pt12", "umf002", "pt07", "umf003", "jh101", "jh108", "pt14"]

for patienti in lists_patients:

    print("+++++++++++++++++++++++++++++++++++++")
    print(patienti)
    print("+++++++++++++++++++++++++++++++++++++")

    # Load a .MAT file for the electrode list
    tmp_folder_path = f'E:\\{patienti}\\3_segmented' # Don't need to change every time
    matfile = [filei for filei in listdir(tmp_folder_path) if filei.endswith(".mat")][0]
    tmp_eeg_file_name =  matfile
    tmp_file_path = tmp_folder_path + '\\' + tmp_eeg_file_name
    print(tmp_file_path)

    eeg = sio.loadmat(tmp_file_path)
    labels = [labi.strip() for labi in eeg['labels'][0][0]]
    for i,labi in enumerate(labels):
        if labi.endswith("'"):
            labi = labi[:-1]
            labels[i] = labi
    regions = []
    [regions.append(split_label(regi)[0]) for regi in labels]
    regions = list(set(regions))

    # print(f"Assign colors to different channels in {regions}")
    possible_colors = ["#Ff0000", "#Ff8c00", "#5cbb38", "#44a3d5", "#005aff", "#9d00ff", "#D944e0", "#Ff007e", "#30812b", "#6f812b", "#643fa6", "#A63f7b","#4f527d","#0583F2","#F24405","#BF5454","#03588C","#025940","#026873","#023873","#715113"]
    brainMap_colors = {}
    for i, regi in enumerate(regions):
        if i >= len(possible_colors): # Not enough colors available
            print("No enough colors available, please add new colors if you don't want repetition of colors")
            brainMap_colors[regi] = possible_colors[i-len(possible_colors)]
        else:
            brainMap_colors[regi] = possible_colors[i]
    # Print in JSON format
    print("{", end="")
    for i, (k,v) in enumerate(brainMap_colors.items()):
        if i != len(brainMap_colors)-1:
            print(f'"{k}":"{v}", ', end='')
        else:
            print(f'"{k}":"{v}"', end='')
    print("}")



+++++++++++++++++++++++++++++++++++++
jh102
+++++++++++++++++++++++++++++++++++++
E:\jh102\3_segmented\sub-jh102_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.mat
{"LSF":"#Ff0000", "RPT":"#Ff8c00", "LAT":"#5cbb38", "RMF":"#44a3d5", "LMF":"#005aff", "RTF":"#9d00ff", "LTF":"#D944e0", "LAF":"#Ff007e", "LPT":"#30812b", "RAF":"#6f812b", "LBT":"#643fa6", "RAT":"#A63f7b", "RSF":"#4f527d", "RBT":"#0583F2", "LPF":"#F24405", "RPF":"#BF5454"}
+++++++++++++++++++++++++++++++++++++
jh104
+++++++++++++++++++++++++++++++++++++
E:\jh104\3_segmented\sub-jh104_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.mat
{"MBT":"#Ff0000", "LAT":"#Ff8c00", "PBT":"#5cbb38", "LMF":"#44a3d5", "LAD":"#005aff", "LFP":"#9d00ff", "LAF":"#D944e0", "LPT":"#Ff007e", "ABT":"#30812b", "LHD":"#6f812b", "LPF":"#643fa6"}
+++++++++++++++++++++++++++++++++++++
ummc001
+++++++++++++++++++++++++++++++++++++
E:\ummc001\3_segmented\sub-ummc001_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.mat
{"GA":"#Ff0000", "F":"#Ff8c00", "GP":"#5c